In [2]:
import numpy as np
import pandas as pd

Verificare se il delta G delle mutazioni ridondanti è uguale o meno. 


In [3]:
# Read the file with pandas
skempi_file = pd.read_csv('C:/Users/Lorenzo/Desktop/Tesi/SKEMPI/skempi_v2.csv', sep = ';') #it is separeted by a semicolon

# Select only single point mutations
df_single_mutation_skempi = skempi_file[skempi_file['Mutation(s)_PDB'].apply(lambda x: len(x.split(',')) == 1)]

# Step 1: Reset index if needed to ensure it's unique
df_single_mutation_skempi.reset_index(inplace=True, drop=True)

# Assuming df_single_mutation_skempi is a subset of another DataFrame
# Replace '298(assumed)' with '298' using .loc
df_single_mutation_skempi.loc[:, 'Temperature'] = df_single_mutation_skempi['Temperature'].replace('298(assumed)', '298')

# Show the first 20 columns
df_single_mutation_skempi.iloc[:, 0:20]

,#Pdb,Mutation(s)_PDB,Mutation(s)_cleaned,iMutation_Location(s),Hold_out_type,Hold_out_proteins,Affinity_mut (M),Affinity_mut_parsed,Affinity_wt (M),Affinity_wt_parsed,Reference,Protein 1,Protein 2,Temperature,kon_mut (M^(-1)s^(-1)),kon_mut_parsed,kon_wt (M^(-1)s^(-1)),kon_wt_parsed,koff_mut (s^(-1)),koff_mut_parsed
0,1CSE_E_I,LI45G,LI38G,COR,Pr/PI,Pr/PI,5.26E-11,5.260000e-11,1.12E-12,1.120000e-12,9048543,Subtilisin Carlsberg,Eglin c,294,NaN,NaN,NaN,NaN,NaN,NaN
1,1CSE_E_I,LI45S,LI38S,COR,Pr/PI,Pr/PI,8.33E-12,8.330000e-12,1.12E-12,1.120000e-12,9048543,Subtilisin Carlsberg,Eglin c,294,NaN,NaN,NaN,NaN,NaN,NaN
2,1CSE_E_I,LI45P,LI38P,COR,Pr/PI,Pr/PI,1.02E-07,1.020000e-07,1.12E-12,1.120000e-12,9048543,Subtilisin Carlsberg,Eglin c,294,NaN,NaN,NaN,NaN,NaN,NaN
3,1CSE_E_I,LI45I,LI38I,COR,Pr/PI,Pr/PI,1.72E-10,1.720000e-10,1.12E-12,1.120000e-12,9048543,Subtilisin Carlsberg,Eglin c,294,NaN,NaN,NaN,NaN,NaN,NaN
4,1CSE_E_I,LI45D,LI38D,COR,Pr/PI,Pr/PI,1.92E-09,1.920000e-09,1.12E-12,1.120000e-12,9048543,Subtilisin Carlsberg,Eglin c,294,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5107,3QIB_ABP_CD,KP9A,KP8A,COR,TCR/pMHC,"TCR/pMHC,1JCK_A_B",>1.1E-03,1.100000e-03,5.5E-06,5.500000e-06,12152083,I-Ek plus MCC peptide,2B4 TCR,298,NaN,NaN,NaN,NaN,NaN,NaN
5108,3QIB_ABP_CD,KP9R,KP8R,COR,TCR/pMHC,"TCR/pMHC,1JCK_A_B",2.4E-04,2.400000e-04,5.5E-06,5.500000e-06,12152083,I-Ek plus MCC peptide,2B4 TCR,298,2070.0,2070.0,4000.0,4000.0,5E-01,0.500
5109,3QIB_ABP_CD,TP12A,TP11A,COR,TCR/pMHC,"TCR/pMHC,1JCK_A_B",>1.1E-03,1.100000e-03,5.5E-06,5.500000e-06,12152083,I-Ek plus MCC peptide,2B4 TCR,298,NaN,NaN,NaN,NaN,NaN,NaN
5110,3QIB_ABP_CD,TP12S,TP11S,COR,TCR/pMHC,"TCR/pMHC,1JCK_A_B",3.38E-05,3.380000e-05,5.5E-06,5.500000e-06,12152083,I-Ek plus MCC peptide,2B4 TCR,298,3960.0,3960.0,4000.0,4000.0,1.34E-01,0.134


In [4]:
# forse vanno rimosse le "assumed" nelle temperature 
# e sistemata la temperatura, non vano aggiunti i 273.15

# Constants for the ΔΔG calculation
R = 1.987 # cal/(mol·K)

# Function to calculate ΔΔG, taking temperature from the dataset
def calculate_delta_delta_G(Kd_mut, Kd_wt, temperature):
    """Calculate ΔΔG using the provided dissociation constants and temperature."""
    try:
        T_kelvin = float(temperature) # Convert temperature to Kelvin (non è necessario aggiungerlo)
        return R * T_kelvin * np.log(Kd_mut / Kd_wt) / 1000  # Convert to kcal/mol
    except ValueError:
        return np.nan  # Return NaN for invalid temperature values

# Filter out rows where necessary affinity data or temperature is missing
skempi_data_filtered = df_single_mutation_skempi.dropna(subset=['Affinity_mut_parsed', 'Affinity_wt_parsed', 'Temperature'])

# Instead of directly assigning the new values, use .loc
skempi_data_filtered.loc[:, 'Temperature'] = pd.to_numeric(skempi_data_filtered['Temperature'], errors='coerce')

# Recalculate ΔΔG for each mutation using .loc
skempi_data_filtered.loc[:, 'DeltaDeltaG'] = skempi_data_filtered.apply(
    lambda row: calculate_delta_delta_G(row['Affinity_mut_parsed'], row['Affinity_wt_parsed'], row['Temperature']), axis=1)

# Remove rows where DeltaDeltaG is NaN
skempi_data_filtered = skempi_data_filtered.dropna(subset=['DeltaDeltaG'])


# Display the first few rows with the calculated ΔΔG
skempi_data_filtered[['Mutation(s)_cleaned', 'Affinity_mut_parsed', 'Affinity_wt_parsed', 'Temperature', 'DeltaDeltaG']]

C:\Users\Lorenzo\AppData\Local\Temp\ipykernel_25056\209850251.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  skempi_data_filtered.loc[:, 'DeltaDeltaG'] = skempi_data_filtered.apply(


,Mutation(s)_cleaned,Affinity_mut_parsed,Affinity_wt_parsed,Temperature,DeltaDeltaG
0,LI38G,5.260000e-11,1.120000e-12,294,2.248727
1,LI38S,8.330000e-12,1.120000e-12,294,1.172173
2,LI38P,1.020000e-07,1.120000e-12,294,6.670962
3,LI38I,1.720000e-10,1.120000e-12,294,2.940849
4,LI38D,1.920000e-09,1.120000e-12,294,4.350229
...,...,...,...,...,...
5107,KP8A,1.100000e-03,5.500000e-06,298,3.137271
5108,KP8R,2.400000e-04,5.500000e-06,298,2.235803
5109,TP11A,1.100000e-03,5.500000e-06,298,3.137271
5110,TP11S,3.380000e-05,5.500000e-06,298,1.075131


## Encode the mutations

In [4]:
#Neural_network_amino_acids
###########

import numpy as np
import pandas as pd

# Assuming df_single_mutation_skempi is your DataFrame

def one_hot_encode_mutation(mutation, amino_acids_order):
    """
    One-hot encode a single amino acid mutation.
    The original amino acid is encoded as -1, the mutated amino acid as +1, and others as 0.
    """
    # Initialize the vector with zeros
    encoding = np.zeros(len(amino_acids_order))

    # Extract the original and mutated amino acids from the mutation string
    original_aa = mutation[0]
    mutated_aa = mutation[-1]

    # Set -1 for the original amino acid and +1 for the mutated amino acid
    encoding[amino_acids_order.index(original_aa)] = -1
    encoding[amino_acids_order.index(mutated_aa)] = 1

    return encoding

# Define the order of the 20 standard amino acids alphabetically
amino_acids_order = sorted(list('ACDEFGHIKLMNPQRSTVWY'))

# Apply the encoding to the 'Mutation(s)_cleaned' column and convert to a numpy array
encoded_mutations = np.array([one_hot_encode_mutation(mutation, amino_acids_order) 
                              for mutation in skempi_data_filtered['Mutation(s)_cleaned']])

# Show the amino acids order and the first few encoded mutations
print("Amino Acids Order:", amino_acids_order)
print("\nFirst few encoded mutations:")
print(encoded_mutations[:5])  # Showing the first 5 encoded mutations


Amino Acids Order: ['A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y']

First few encoded mutations:
[[ 0.  0.  0.  0.  0.  1.  0.  0.  0. -1.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0. -1.  0.  0.  0.  0.  0.  1.  0.  0.
   0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0. -1.  0.  0.  1.  0.  0.  0.  0.  0.
   0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  1.  0. -1.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.]
 [ 0.  0.  1.  0.  0.  0.  0.  0.  0. -1.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.]]


## Encode the environments: 20 elements vector

In [28]:
import pandas as pd
import os
import glob




def read_contacts(file_path):
    """
    Read a contact file and return a list of amino acids
    """
    with open(file_path, 'r') as file:
        lines = file.readlines()

    # Extracting amino acid contacts
    contacts = []
    for line in lines[3:]:  # Skipping header lines
        parts = line.split()
        if len(parts) >= 2:
            aa = parts[0]  # Amino acid type
            contacts.append(aa)
    return contacts

def three_to_one_letter(three_letter_code):
    """
    Convert a three-letter amino acid code to a one-letter code
    """
    conversion_dict = {
        'ALA': 'A', 'CYS': 'C', 'ASP': 'D', 'GLU': 'E', 'PHE': 'F', 'GLY': 'G', 'HIS': 'H', 
        'ILE': 'I', 'LYS': 'K', 'LEU': 'L', 'MET': 'M', 'ASN': 'N', 'PRO': 'P', 'GLN': 'Q', 
        'ARG': 'R', 'SER': 'S', 'THR': 'T', 'VAL': 'V', 'TRP': 'W', 'TYR': 'Y'
    }
    return conversion_dict.get(three_letter_code, '')

def calculate_aa_difference(original_contacts, mutant_contacts):
    """
    Calculate the difference in amino acid counts between original and mutant contacts.
    Positive values indicate more amino acids in the mutant list,
    negative values indicate less.
    """
    # One-letter representation of amino acids in alphabetical order
    aa_dict = {'A': 0, 'C': 1, 'D': 2, 'E': 3, 'F': 4, 'G': 5, 'H': 6, 
               'I': 7, 'K': 8, 'L': 9, 'M': 10, 'N': 11, 'P': 12, 'Q': 13, 
               'R': 14, 'S': 15, 'T': 16, 'V': 17, 'W': 18, 'Y': 19}

    # Initialize count vectors
    original_vector = [0] * 20
    mutant_vector = [0] * 20

    # Count amino acids in original and mutant lists
    for aa in original_contacts:
        aa_one_letter = three_to_one_letter(aa)
        if aa_one_letter in aa_dict:
            original_vector[aa_dict[aa_one_letter]] += 1

    for aa in mutant_contacts:
        aa_one_letter = three_to_one_letter(aa)
        if aa_one_letter in aa_dict:
            mutant_vector[aa_dict[aa_one_letter]] += 1

    # Calculate the difference
    difference_vector = [mut - orig for mut, orig in zip(mutant_vector, original_vector)]
    return difference_vector

################


import numpy as np

def process_skempi_dataset_20(df_single_mutation_skempi, original_contacts_path, mutant_contacts_path):
    """
    Process the SKEMPI dataset and contact files using the provided DataFrame,
    considering different file patterns for original and mutant files.
    Stores the difference vectors in a NumPy array suitable for machine learning.
    """
    vectors = []  # List to store the difference vectors
    file_names = []  # List to store the corresponding file names

    for index, row in df_single_mutation_skempi.iterrows():
        # Extracting the PDB ID and mutation
        pdb_id = row['#Pdb'].split('_')[0]  # Adjusted to use the correct column name '#Pdb'
        mutation = row['Mutation(s)_cleaned']

        # File patterns for original and mutant files
        original_file_pattern = f"{pdb_id}_{mutation}_contacts.txt"
        mutant_file_pattern = f"{pdb_id}_{mutation}_*_contacts.txt"

        # Finding all matching files
        original_files = glob.glob(os.path.join(original_contacts_path, original_file_pattern))
        mutant_files = glob.glob(os.path.join(mutant_contacts_path, mutant_file_pattern))
        print(mutation)
        for orig_file, mut_file in zip(original_files, mutant_files):
            if os.path.exists(orig_file) and os.path.exists(mut_file):
                original_contacts = read_contacts(orig_file)
                mutant_contacts = read_contacts(mut_file)

                # Calculate the difference in amino acid counts
                diff_contacts = calculate_aa_difference(original_contacts, mutant_contacts)

                # Storing the result
                vectors.append(diff_contacts)
                file_names.append(os.path.basename(orig_file))

    # Converting the list of vectors to a NumPy array
    vectors_np = np.array(vectors)

    return vectors_np, file_names

# Example usage (assuming df_single_mutation_skempi is already defined)
environment_amino_acids, file_names = process_skempi_dataset_20(skempi_data_filtered, 
                                            'C:/Users/Lorenzo/Desktop/Tesi/SKEMPI/original_aminoacid_contacts',
                                            'C:/Users/Lorenzo/Desktop/Tesi/SKEMPI/mutant_aminoacid_contacts')

# You can now use `vectors` for training your neural network in Scikit-learn




LI38G
LI38S
LI38P
LI38I
LI38D
LI38E
LI38G
LI38S
LI38P
LI38I
LI38D
LI38E
RI38K
KI46R
YI42A
YI42G
RI46A
RI48A
RI48C
TI39D
TI39A
EI41A
VI51A
AI39T
PI39T
AI41E
SI41E
AI43R
AI46R
AI48R
TI39A
TI39P
EI41A
EI41S
RI43A
RI46A
RI48A
GI40M
AI40M
KI40M
YI40M
FI40M
AI42Y
MI40G
MI40A
MI40K
MI40Y
MI40F
YI42A
MI67E
MI67D
MI67H
MI67K
MI67R
MI67G
MI67A
MI67L
MI67V
MI67I
IA5A
IA5R
TA6A
TA6D
QA8A
QA8R
EA9Q
IA11A
KA12S
KA12E
TA13A
TA13D
NA15A
NA15D
SA16A
SA16D
EA19A
EA19R
RA53Q
KA77A
KA77E
QA78A
QA78E
RA81A
RA81E
FA82A
FA82D
KA84A
KA84D
RA85A
RA85E
RA88Q
RA88A
NA89A
WA91A
WA91D
KA25A
RA57A
RA81Q
RA85A
HA100A
YD29F
YD29A
DD35A
WD38F
DD39A
TD42A
WD44F
ED74A
ED78A
EA71Q
EA71W
EA71Y
EA71F
EA71C
EA71S
EA71A
RA57K
RA57A
HA100Q
HA100D
HA100G
HA100A
HA100L
AA27K
AD36D
AA27K
AD43T
AA102H
FD30Y
DD35A
DD39A
ED74A
ED78A
AD81E
AD76E
KA25A
WA33F
DA52A
NA56A
RA57A
EA58A
EA71A
RA85A
HA100A
KA25A
WA33F
DA52A
NA56A
RA57A
EA58A
EA71A
HA100A
LB20T
VB26Y
YB91V
TB20A
YB26A
NB60A
YB90A
VB91A
KB103A
FB176A
LB20T
VB26Y
YB91V
TB20A


KeyboardInterrupt: 

### Version with vectors that have 40 elements

In [ ]:
import pandas as pd
import os
import glob
import numpy as np

def read_contacts(file_path):
    """
    Read a contact file and return a list of amino acids
    """
    with open(file_path, 'r') as file:
        lines = file.readlines()

    # Extracting amino acid contacts
    contacts = []
    for line in lines[3:]:  # Skipping header lines
        parts = line.split()
        if len(parts) >= 2:
            aa = parts[0]  # Amino acid type
            contacts.append(aa)
    return contacts

def three_to_one_letter(three_letter_code):
    """
    Convert a three-letter amino acid code to a one-letter code
    """
    conversion_dict = {
        'ALA': 'A', 'CYS': 'C', 'ASP': 'D', 'GLU': 'E', 'PHE': 'F', 'GLY': 'G', 'HIS': 'H', 
        'ILE': 'I', 'LYS': 'K', 'LEU': 'L', 'MET': 'M', 'ASN': 'N', 'PRO': 'P', 'GLN': 'Q', 
        'ARG': 'R', 'SER': 'S', 'THR': 'T', 'VAL': 'V', 'TRP': 'W', 'TYR': 'Y'
    }
    return conversion_dict.get(three_letter_code, '')

def calculate_aa_counts(contacts):
    """
    Calculate the counts of each amino acid in a list of contacts.
    Returns a vector of length 20 representing the counts of each amino acid.
    """
    aa_dict = {'A': 0, 'C': 1, 'D': 2, 'E': 3, 'F': 4, 'G': 5, 'H': 6, 
               'I': 7, 'K': 8, 'L': 9, 'M': 10, 'N': 11, 'P': 12, 'Q': 13, 
               'R': 14, 'S': 15, 'T': 16, 'V': 17, 'W': 18, 'Y': 19}

    count_vector = [0] * 20
    for aa in contacts:
        aa_one_letter = three_to_one_letter(aa)
        if aa_one_letter in aa_dict:
            count_vector[aa_dict[aa_one_letter]] += 1
    return count_vector

def process_skempi_dataset_40(df_single_mutation_skempi, original_contacts_path, mutant_contacts_path):
    """
    Process the SKEMPI dataset and contact files using the provided DataFrame,
    considering different file patterns for original and mutant files.
    Concatenates the amino acid count vectors for wild type and mutant into a single vector of 40 elements.
    """
    vectors = []  # List to store the concatenated vectors
    file_names = []  # List to store the corresponding file names

    for index, row in df_single_mutation_skempi.iterrows():
        # Extracting the PDB ID and mutation
        pdb_id = row['#Pdb'].split('_')[0]  # Adjusted to use the correct column name '#Pdb'
        mutation = row['Mutation(s)_cleaned']

        # File patterns for original and mutant files
        original_file_pattern = f"{pdb_id}_{mutation}_contacts.txt"
        mutant_file_pattern = f"{pdb_id}_{mutation}_*_contacts.txt"

        # Finding all matching files
        original_files = glob.glob(os.path.join(original_contacts_path, original_file_pattern))
        mutant_files = glob.glob(os.path.join(mutant_contacts_path, mutant_file_pattern))

        for orig_file, mut_file in zip(original_files, mutant_files):
            if os.path.exists(orig_file) and os.path.exists(mut_file):
                original_contacts = read_contacts(orig_file)
                mutant_contacts = read_contacts(mut_file)

                # Count amino acids in original and mutant lists
                original_vector = calculate_aa_counts(original_contacts)
                mutant_vector = calculate_aa_counts(mutant_contacts)

                # Concatenating the vectors
                concat_vector = original_vector + mutant_vector

                # Storing the result
                vectors.append(concat_vector)
                file_names.append(os.path.basename(orig_file))

    # Converting the list of vectors to a NumPy array
    vectors_np = np.array(vectors)

    return vectors_np, file_names

# Example usage (assuming df_single_mutation_skempi is already defined)
#ectors, file_names = process_skempi_dataset(skempi_data_filtered, 
#                                           'C:/Users/Lorenzo/Desktop/Tesi/SKEMPI/original_aminoacid_contacts',
#                                         'C:/Users/Lorenzo/Desktop/Tesi/SKEMPI/mutant_aminoacid_contacts')


### Vectors for RSA

In [7]:

# Normalization values for RSA calculation
Norm_Acc = {
    "A": 106.0, "C": 135.0, "D": 163.0, "E": 194.0, "F": 197.0,
    "G": 84.0, "H": 184.0, "I": 169.0, "K": 205.0, "L": 164.0,
    "M": 188.0, "N": 157.0, "P": 136.0, "Q": 198.0, "R": 248.0,
    "S": 130.0, "T": 142.0, "V": 142.0, "W": 227.0, "Y": 222.0,
    "B": 160.0, "X": 180.0, "Z": 196.0  # Additional values for B, X, Z
}


#############

def calculate_rsa(dssp_file, residue_id, chain_id, aa_type):
    with open(dssp_file, 'r') as file:
        lines = file.readlines()

    start_reading = False
    for line in lines:
        if start_reading:
            if len(line) > 10:
                try:
                    dssp_residue_id = int(line[5:10].strip())
                    dssp_aa = line[13].strip()
                    dssp_chain_id = line[11].strip()

                    # Debugging print
                    # print(f"Line Info - Residue: {dssp_residue_id}, Chain: {dssp_chain_id}, AA: {dssp_aa}")

                    if dssp_residue_id == residue_id and dssp_chain_id == chain_id and dssp_aa == aa_type:
                        acc_str = line[35:38].strip()
                        if acc_str.isdigit():
                            dssp_acc = float(acc_str)
                            rsa = dssp_acc / Norm_Acc.get(dssp_aa, 1)
                            rsa = min(rsa, 1)
                            return rsa
                except ValueError as e:
                    continue
                    #print(f"Error processing line: {line}. Error: {e}")

        elif "#" in line and "RESIDUE" in line and "CHAIN" in line:
            start_reading = True

    return None


#####################

#With this function it should all be fine

def rsa_process_skempi_dataset(df_single_mutation_skempi, dssp_folders):
    rsa_vectors = []
    mutation_count = {}

    for index, row in df_single_mutation_skempi.iterrows():
        pdb_id = row['#Pdb'].split('_')[0]
        mutation = row['Mutation(s)_cleaned']
        original_aa, chain_id, residue_position, mutant_aa = mutation[0], mutation[1], int(mutation[2:-1]), mutation[-1]

        # Increment mutation count
        mutation_key = f"{pdb_id}_{mutation}"
        if mutation_key not in mutation_count:
            mutation_count[mutation_key] = 0
        mutation_count[mutation_key] += 1

        rsa_values = []
        for condition in ['wild_type_complex', 'wild_type_monomer', 'mutant_complex', 'mutant_monomer']:
            folder = dssp_folders[condition]

            # Construct file pattern based on condition and mutation count
            if 'wild_type_monomer' in condition:
                dssp_file_pattern = f"{pdb_id}_{chain_id}.dssp"
            elif 'wild_type_complex' in condition:
                dssp_file_pattern = f"{pdb_id}.dssp"
            elif 'mutant_monomer' in condition:
                count = mutation_count[mutation_key]
                dssp_file_pattern = f"{pdb_id}_{mutation}_{chain_id}_chain.dssp" if count == 1 else f"{pdb_id}_{mutation}_duplicate_{count - 1}_{chain_id}_chain.dssp"
            elif 'mutant_complex' in condition:
                count = mutation_count[mutation_key]
                dssp_file_pattern = f"{pdb_id}_{mutation}.dssp" if count == 1 else f"{pdb_id}_{mutation}_duplicate_{count - 1}.dssp"

            # print(f"Looking for files in {folder} with pattern: {dssp_file_pattern}")


            dssp_files = glob.glob(os.path.join(folder, dssp_file_pattern))

            if dssp_files:
                dssp_file = dssp_files[0]

                # print(f"Selected file for condition '{condition}': {dssp_file}")

                aa_to_check = mutant_aa if 'mutant' in condition else original_aa
                rsa = calculate_rsa(dssp_file, residue_position, chain_id, aa_to_check)
                if rsa is not None:
                    rsa_values.append(rsa)
            else:
                print(f"No files found for pattern: {dssp_file_pattern}")

        if len(rsa_values) == 4:
            rsa_vectors.append(rsa_values)
        else:
            print(f"Missing RSA values for PDB ID: {pdb_id}, Mutation: {mutation}")

    return rsa_vectors


# Folder paths
dssp_folders = {
    'wild_type_complex': 'C:/Users/Lorenzo/Desktop/Tesi/SKEMPI/dssp/dssp_wild_type_complexes_output',
    'wild_type_monomer': 'C:/Users/Lorenzo/Desktop/Tesi/SKEMPI/dssp/dssp_wild_type_monomers_output',
    'mutant_complex': 'C:/Users/Lorenzo/Desktop/Tesi/SKEMPI/dssp/dssp_mutant_complexes_output',
    'mutant_monomer': 'C:/Users/Lorenzo/Desktop/Tesi/SKEMPI/dssp/dssp_mutant_monomers_output'
}

# Example usage (assuming df_single_mutation_skempi is already defined)
# rsa_vectors = rsa_process_skempi_dataset(skempi_data_filtered, dssp_folders)

# Now rsa_vectors contains the list of RSA vectors for machine learning



## Split in 10 groups with hold_out_protein

In [8]:
grouped_type= skempi_data_filtered.groupby(['Hold_out_proteins']).size()
sorted_grouped_type = grouped_type.sort_values(ascending=False)
sorted_grouped_type

Hold_out_proteins
Pr/PI                1026
AB/AG                 719
TCR/pMHC,1JCK_A_B     454
3BT1_A_U              245
1A22_A_B,1BP3_A_B     212
                     ... 
4EKD_A_B                1
1YCS_A_B                1
4MYW_A_B,3U82_A_B       1
4LRX_AB_CD              1
3RF3_A_C                1
Length: 154, dtype: int64

In [9]:
import pandas as pd

# Assuming skempi_data_filtered is your DataFrame
# First, get the counts of each type as you did before
grouped_type = skempi_data_filtered.groupby(['Hold_out_proteins']).size()
sorted_grouped_type = grouped_type.sort_values(ascending=False)

# Initialize the groups
groups = {i: [] for i in range(10)}
group_counts = {i: 0 for i in range(10)}

# Assign a group to each Hold_out_proteins type
type_to_group = {}
for protein_type in sorted_grouped_type.index:
    min_group = min(group_counts, key=group_counts.get)
    type_to_group[protein_type] = min_group
    group_counts[min_group] += sorted_grouped_type[protein_type]

# Now iterate over the DataFrame and assign each row to the appropriate group
for index, row in skempi_data_filtered.iterrows():
    group_id = type_to_group[row['Hold_out_proteins']]
    groups[group_id].append(row)

# Convert each group to a DataFrame and store in group_dataframes with "Group N" as keys
grouped_dfs_hold_out = {f"Group {group_id + 1}": pd.DataFrame(group) for group_id, group in groups.items()}

# Print the results
for group_name, group_df in grouped_dfs_hold_out.items():
    print(f"{group_name}:")
    print(f"Number of elements: {len(group_df)}")
   # print(f"Types: {group_df['Hold_out_proteins'].unique()}")
    print()


Group 1:
Number of elements: 1026

Group 2:
Number of elements: 719

Group 3:
Number of elements: 454

Group 4:
Number of elements: 394

Group 5:
Number of elements: 394

Group 6:
Number of elements: 393

Group 7:
Number of elements: 393

Group 8:
Number of elements: 393

Group 9:
Number of elements: 393

Group 10:
Number of elements: 393



In [10]:
grouped_dfs_hold_out

{'Group 1':           #Pdb Mutation(s)_PDB Mutation(s)_cleaned iMutation_Location(s)   
 0     1CSE_E_I           LI45G               LI38G                   COR  \
 1     1CSE_E_I           LI45S               LI38S                   COR   
 2     1CSE_E_I           LI45P               LI38P                   COR   
 3     1CSE_E_I           LI45I               LI38I                   COR   
 4     1CSE_E_I           LI45D               LI38D                   COR   
 ...        ...             ...                 ...                   ...   
 3117  3BX1_A_C           AC86T               AC86T                   SUP   
 3118  3BX1_A_C           YC87I               YC87I                   COR   
 3119  3BX1_A_C           YC87A               YC87A                   COR   
 3120  3BX1_A_C           TC88V               TC88V                   COR   
 3121  3BX1_A_C           TC89A               TC89A                   COR   
 
      Hold_out_type Hold_out_proteins Affinity_mut (M)  Affinit

## Vector mutation for each group

In [11]:
# Assuming you have a function 'one_hot_encode_mutation' defined somewhere that encodes the mutations
# Define the order of the 20 standard amino acids alphabetically
amino_acids_order = sorted(list('ACDEFGHIKLMNPQRSTVWY'))

# Dictionary to store encoded mutations for each group
group_encoded_mutations_hold_out = {}

for group_name, group_df in grouped_dfs_hold_out.items():
    # Apply the encoding to the 'Mutation(s)_cleaned' column and convert to a numpy array
    encoded_mutations = np.array([one_hot_encode_mutation(mutation, amino_acids_order) 
                                  for mutation in group_df['Mutation(s)_cleaned']])
    group_encoded_mutations_hold_out[group_name] = encoded_mutations

# Now you have the encoded mutations for each group stored in group_encoded_mutations_hold_out

# To access and view the encoded mutations for each group
print("Amino Acids Order:", amino_acids_order)
for group_name, encoded_mutations in group_encoded_mutations_hold_out.items():
    print(f"\nEncoded mutations for {group_name}:")
    print(encoded_mutations[:5])  # Showing the first 5 encoded mutations for each group


Amino Acids Order: ['A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y']

Encoded mutations for Group 1:
[[ 0.  0.  0.  0.  0.  1.  0.  0.  0. -1.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0. -1.  0.  0.  0.  0.  0.  1.  0.  0.
   0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0. -1.  0.  0.  1.  0.  0.  0.  0.  0.
   0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  1.  0. -1.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.]
 [ 0.  0.  1.  0.  0.  0.  0.  0.  0. -1.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.]]

Encoded mutations for Group 2:
[[ 1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0. -1.]
 [ 1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0. -1.]
 [ 1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. -1.  0.
   0.  0.]
 [ 1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. -1.  0.
   0.  0.]
 [ 1.  0.  0.  0.  0.  0.  0.  0.  0. -1.  0.  0. 

## Vector environment for each group

In [13]:
import numpy as np
import pandas as pd
import os
import glob

# Paths to the directories containing the contact files
original_contacts_path = 'C:/Users/Lorenzo/Desktop/Tesi/SKEMPI/original_aminoacid_contacts'
mutant_contacts_path = 'C:/Users/Lorenzo/Desktop/Tesi/SKEMPI/mutant_aminoacid_contacts'

# Dictionary to store the processed data for each group
group_processed_data_hold_out = {}

for group_name, group_df in grouped_dfs_hold_out.items():
    vectors, file_names = process_skempi_dataset_20(group_df, original_contacts_path, mutant_contacts_path)
    group_processed_data_hold_out[group_name] = vectors
                                        
# Now group_processed_data contains the processed data for each group

# Example: Accessing processed data for 'Group_1'

## Vector RSA for each group

In [14]:
# Import necessary modules
import pandas as pd
import numpy as np
import os
import glob

# Define your functions: calculate_rsa, rsa_process_skempi_dataset
# ... [Your functions' definitions go here] ...

# Folder paths for DSSP files
dssp_folders = {
    'wild_type_complex': 'C:/Users/Lorenzo/Desktop/Tesi/SKEMPI/dssp/dssp_wild_type_complexes_output',
    'wild_type_monomer': 'C:/Users/Lorenzo/Desktop/Tesi/SKEMPI/dssp/dssp_wild_type_monomers_output',
    'mutant_complex': 'C:/Users/Lorenzo/Desktop/Tesi/SKEMPI/dssp/dssp_mutant_complexes_output',
    'mutant_monomer': 'C:/Users/Lorenzo/Desktop/Tesi/SKEMPI/dssp/dssp_mutant_monomers_output'
}

# Dictionary to store the RSA vectors for each group
group_rsa_vectors_hold_out = {}

for group_name, group_df in grouped_dfs_hold_out.items():
    rsa_vectors = rsa_process_skempi_dataset(group_df, dssp_folders)
    rsa_vectors_array = np.array(rsa_vectors)
    group_rsa_vectors_hold_out[group_name] = rsa_vectors_array

# Now group_rsa_vectors contains the RSA vectors for each group

# Example: Accessing RSA vectors for 'Group_1'
    print(f"RSA vectors for {group_name}:")
    print(group_rsa_vectors_hold_out[group_name])

RSA vectors for Group 1:
[[0.09146341 1.         0.16666667 0.89285714]
 [0.09146341 1.         0.06923077 0.96153846]
 [0.09146341 1.         0.06617647 0.94852941]
 ...
 [0.         0.91441441 0.05660377 0.91509434]
 [0.01408451 0.26056338 0.02112676 0.27464789]
 [0.08450704 0.8943662  0.14150943 0.9245283 ]]
RSA vectors for Group 2:
[[0.00900901 0.30630631 0.12264151 0.16981132]
 [0.04504505 0.0990991  0.0754717  0.0754717 ]
 [0.03521127 0.54929577 0.03773585 0.58490566]
 ...
 [0.13414634 0.82317073 0.17924528 0.62264151]
 [0.10309278 0.68556701 0.13207547 0.72641509]
 [0.27411168 0.42639594 0.26415094 0.26415094]]
RSA vectors for Group 3:
[[0.         0.11320755 0.         0.09230769]
 [0.04907975 0.33742331 0.05660377 0.21698113]
 [0.04907975 0.33742331 0.06153846 0.27692308]
 ...
 [0.06338028 1.         0.06603774 0.98113208]
 [0.06338028 1.         0.04615385 0.93076923]
 [0.06338028 1.         0.07643312 0.95541401]]
RSA vectors for Group 4:
[[0.24203822 0.73248408 0.36956522 0

## Merging of vectors

In [16]:
# Initialize the dictionary for the final combined arrays
final_group_arrays_hold_out = {}

# Ensure all dictionaries have the same group keys
group_names = group_encoded_mutations_hold_out.keys()

for group_name in group_names:
    # Get the arrays from each dictionary for the current group
    encoded_mutations_array = group_encoded_mutations_hold_out[group_name]
    processed_data_array = group_processed_data_hold_out[group_name]
    rsa_vectors_array = group_rsa_vectors_hold_out[group_name]
    
    # Ensure all arrays are numpy arrays (if they aren't already)
    encoded_mutations_array = np.array(encoded_mutations_array)
    processed_data_array = np.array(processed_data_array)
    rsa_vectors_array = np.array(rsa_vectors_array)

    # Check if the number of rows is the same in all arrays
    if (encoded_mutations_array.shape[0] == processed_data_array.shape[0] == rsa_vectors_array.shape[0]):
        # Concatenate the arrays along columns
        combined_array = np.concatenate([encoded_mutations_array, processed_data_array, rsa_vectors_array], axis=1)
    else:
        print(f"Arrays for group {group_name} cannot be concatenated due to mismatched dimensions.")
        continue

    # Store the combined array in the new dictionary
    final_group_arrays_hold_out[group_name] = combined_array



In [17]:
for group_name, combined_array in final_group_arrays_hold_out.items():
    # Get the shape of the array
    array_shape = combined_array.shape
    
    # Print the group name and the dimensions of its array
    print(f"Dimensions of the array for {group_name}: {array_shape}")

Dimensions of the array for Group 1: (1026, 44)
Dimensions of the array for Group 2: (719, 44)
Dimensions of the array for Group 3: (454, 44)
Dimensions of the array for Group 4: (394, 44)
Dimensions of the array for Group 5: (394, 44)
Dimensions of the array for Group 6: (393, 44)
Dimensions of the array for Group 7: (393, 44)
Dimensions of the array for Group 8: (393, 44)
Dimensions of the array for Group 9: (393, 44)
Dimensions of the array for Group 10: (393, 44)


# Training

## Random Forest regression

## Random Forest with best parameters

In [19]:
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Initialize a structure to store the performance of the model on each fold
performance_metrics = {
    'MSE': {},
    'MAE': {},
    'R2': {}
}

for test_group in final_group_arrays_hold_out.keys():
    # Prepare the training data: use all groups except the test_group
    train_data = np.concatenate([array for group, array in final_group_arrays_hold_out.items() if group != test_group])
    train_labels = np.concatenate([grouped_dfs_hold_out[group]['DeltaDeltaG'].values for group in grouped_dfs_hold_out if group != test_group])
    
    # Ensure the train_labels is a flat array
    train_labels = train_labels.ravel()
    
    # Prepare the test data
    test_data = final_group_arrays_hold_out[test_group]
    test_labels = grouped_dfs_hold_out[test_group]['DeltaDeltaG'].values
    
    # Initialize the model with the best parameters
    model = RandomForestRegressor(max_depth=10, n_estimators=200, random_state=42)
    
    # Train the model
    model.fit(train_data, train_labels)
    
    # Make predictions on the test set
    predictions = model.predict(test_data)
    
    # Calculate the performance metrics
    mse = mean_squared_error(test_labels, predictions)
    mae = mean_absolute_error(test_labels, predictions)
    r2 = r2_score(test_labels, predictions)
    
    # Store the performance metrics
    performance_metrics['MSE'][test_group] = mse
    performance_metrics['MAE'][test_group] = mae
    performance_metrics['R2'][test_group] = r2

# Output the performance metrics for each fold
for metric, values in performance_metrics.items():
    for group, value in values.items():
        print(f"{metric} for test group {group}: {value:.4f}")
    
    # Calculate and print the average performance across all folds for each metric
    average_value = np.mean(list(values.values()))
    print(f"Average {metric} across all folds: {average_value:.4f}")
    print('---')  # Separator for clarity


MSE for test group Group 1: 6.2269
MSE for test group Group 2: 2.0932
MSE for test group Group 3: 1.8673
MSE for test group Group 4: 0.5872
MSE for test group Group 5: 1.2703
MSE for test group Group 6: 1.5744
MSE for test group Group 7: 2.0901
MSE for test group Group 8: 2.4297
MSE for test group Group 9: 1.2123
MSE for test group Group 10: 1.3956
Average MSE across all folds: 2.0747
---
MAE for test group Group 1: 1.7032
MAE for test group Group 2: 1.0100
MAE for test group Group 3: 0.9677
MAE for test group Group 4: 0.5522
MAE for test group Group 5: 0.7732
MAE for test group Group 6: 0.8492
MAE for test group Group 7: 1.0719
MAE for test group Group 8: 1.1036
MAE for test group Group 9: 0.8165
MAE for test group Group 10: 0.8550
Average MAE across all folds: 0.9702
---
R2 for test group Group 1: 0.0447
R2 for test group Group 2: 0.2080
R2 for test group Group 3: -0.1448
R2 for test group Group 4: 0.2656
R2 for test group Group 5: 0.1575
R2 for test group Group 6: 0.2038
R2 for test

Random Forests with R parameter, besides R^2

In [20]:
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, mean_squared_error

# Initialize a structure to store the performance of the model on each fold
performance_metrics = {
    'MSE': {},
    'MAE': {},
    'R2': {},
    'R': {}  # Add R here
}

for test_group in final_group_arrays_hold_out.keys():
    # Prepare the training data: use all groups except the test_group
    train_data = np.concatenate([array for group, array in final_group_arrays_hold_out.items() if group != test_group])
    train_labels = np.concatenate([grouped_dfs_hold_out[group]['DeltaDeltaG'].values for group in grouped_dfs_hold_out if group != test_group])
    
    # Ensure the train_labels is a flat array
    train_labels = train_labels.ravel()
    
    # Prepare the test data
    test_data = final_group_arrays_hold_out[test_group]
    test_labels = grouped_dfs_hold_out[test_group]['DeltaDeltaG'].values
    
    # Initialize the model with the best parameters
    model = RandomForestRegressor(max_depth=10, n_estimators=200, random_state=42)
    
    # Train the model
    model.fit(train_data, train_labels)
    
    # Make predictions on the test set
    predictions = model.predict(test_data)
    
    # Calculate the performance metrics
    mse = mean_squared_error(test_labels, predictions)
    mae = mean_absolute_error(test_labels, predictions)
    r2 = r2_score(test_labels, predictions)
    
    # Calculate R (correlation coefficient) from R^2
    if r2 >= 0:  # Ensure R^2 is not negative
        r = np.sqrt(r2) * (np.sign(np.corrcoef(test_labels, predictions)[0, 1]))
    else:
        r = 0  # Set R to 0 if R^2 is negative, though this case should be rare or non-existent
    
    # Store the performance metrics
    performance_metrics['MSE'][test_group] = mse
    performance_metrics['MAE'][test_group] = mae
    performance_metrics['R2'][test_group] = r2
    performance_metrics['R'][test_group] = r  # Store R here

# Output the performance metrics for each fold
for metric, values in performance_metrics.items():
    for group, value in values.items():
        print(f"{metric} for test group {group}: {value:.4f}")
    
    # Calculate and print the average performance across all folds for each metric
    average_value = np.mean(list(values.values()))
    print(f"Average {metric} across all folds: {average_value:.4f}")
    print('---')  # Separator for clarity


MSE for test group Group 1: 6.2269
MSE for test group Group 2: 2.0932
MSE for test group Group 3: 1.8673
MSE for test group Group 4: 0.5872
MSE for test group Group 5: 1.2703
MSE for test group Group 6: 1.5744
MSE for test group Group 7: 2.0901
MSE for test group Group 8: 2.4297
MSE for test group Group 9: 1.2123
MSE for test group Group 10: 1.3956
Average MSE across all folds: 2.0747
---
MAE for test group Group 1: 1.7032
MAE for test group Group 2: 1.0100
MAE for test group Group 3: 0.9677
MAE for test group Group 4: 0.5522
MAE for test group Group 5: 0.7732
MAE for test group Group 6: 0.8492
MAE for test group Group 7: 1.0719
MAE for test group Group 8: 1.1036
MAE for test group Group 9: 0.8165
MAE for test group Group 10: 0.8550
Average MAE across all folds: 0.9702
---
R2 for test group Group 1: 0.0447
R2 for test group Group 2: 0.2080
R2 for test group Group 3: -0.1448
R2 for test group Group 4: 0.2656
R2 for test group Group 5: 0.1575
R2 for test group Group 6: 0.2038
R2 for test

## Random forests classifier

In [21]:
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.metrics import accuracy_score, auc
from sklearn.metrics import matthews_corrcoef

# Initialize a structure to store the performance of the model on each fold
performance_classification = {
    'ACC': {},
    'MCC': {},
    'AUC': {}
}

for test_group in final_group_arrays_hold_out.keys():
    # Prepare the training data: use all groups except the test_group
    train_data = np.concatenate([array for group, array in final_group_arrays_hold_out.items() if group != test_group])
    train_labels = np.concatenate([grouped_dfs_hold_out[group]['DeltaDeltaG'].values for group in grouped_dfs_hold_out if group != test_group])
    
    # Ensure the train_labels is a flat array
    train_labels = train_labels.ravel()
    
    # Prepare the test data
    test_data = final_group_arrays_hold_out[test_group]
    test_labels = np.sign(grouped_dfs_hold_out[test_group]['DeltaDeltaG'].values) # 2*(test_labels > 0) - 1
    test_labels = 2*(test_labels > 0) - 1 # Transform in +1 and - 1
    # Initialize the model with the best parameters
    model = RandomForestClassifier(max_depth=10, n_estimators=200, random_state=42)
    
    # Train the model
    model.fit(train_data, train_labels)
    
    # Make predictions on the test set
    predictions = model.predict(test_data)
    predictions_probability = model.predict_proba(test_data)

    # Calculate the performance metrics
    # Calcolare overall accuracy and MCC
    accuracy = accuracy_score(test_labels, predictions)
    mcc = matthews_corrcoef(test_labels, predictions)
    area_under_curve = auc (test_labels, predictions_probability)
    #mse = mean_squared_error(test_labels, predictions)
    #mae = mean_absolute_error(test_labels, predictions)
    #r2 = r2_score(test_labels, predictions)
    
    # Store the performance metrics
    performance_classification['ACC'][test_group] = accuracy
    performance_classification['MCC'][test_group] = mcc
    performance_classification['AUC'][test_group] = area_under_curve

# Output the performance metrics for each fold
for metric, values in performance_metrics.items():
    for group, value in values.items():
        print(f"{metric} for test group {group}: {value:.4f}")
    
    # Calculate and print the average performance across all folds for each metric
    average_value = np.mean(list(values.values()))
    print(f"Average {metric} across all folds: {average_value:.4f}")
    print('---')  # Separator for clarity


ValueError: Unknown label type: continuous. Maybe you are trying to fit a classifier, which expects discrete classes on a regression target with continuous values.

In [24]:
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, matthews_corrcoef, roc_auc_score

# Initialize a structure to store the performance of the model on each fold
performance_classification = {
    'ACC': {},
    'MCC': {},
    'AUC': {}
}

for test_group in final_group_arrays_hold_out.keys():
    # Prepare the training data: use all groups except the test_group
    train_data = np.concatenate([array for group, array in final_group_arrays_hold_out.items() if group != test_group])
    train_labels = np.concatenate([grouped_dfs_hold_out[group]['DeltaDeltaG'].values for group in grouped_dfs_hold_out if group != test_group])
    
    # Ensure the train_labels is a flat array and transform to -1 and +1
    train_labels = train_labels.ravel()
    train_labels = np.sign(train_labels)
    train_labels = 2 * (train_labels > 0) - 1  # Transform in +1 and -1
    
    # Prepare the test data and transform labels to -1 and +1
    test_data = final_group_arrays_hold_out[test_group]
    test_labels = np.sign(grouped_dfs_hold_out[test_group]['DeltaDeltaG'].values)
    test_labels = 2 * (test_labels > 0) - 1
    
    # Check unique values in train_labels and test_labels
    #print(f"Unique values in train_labels for group {test_group}:", np.unique(train_labels))
    #print(f"Unique values in test_labels for group {test_group}:", np.unique(test_labels))
    
    # Initialize the model with the best parameters
    model = RandomForestClassifier(max_depth=10, n_estimators=200, random_state=42)
    
    # Train the model
    model.fit(train_data, train_labels)
    
    # Make predictions on the test set
    predictions = model.predict(test_data)
    predictions_probability = model.predict_proba(test_data)[:, 1]  # Get probabilities for the positive class

    # Calculate the performance metrics
    accuracy = accuracy_score(test_labels, predictions)
    mcc = matthews_corrcoef(test_labels, predictions)
    area_under_curve = roc_auc_score(test_labels, predictions_probability)  # Compute AUC using roc_auc_score
    
    # Store the performance metrics
    performance_classification['ACC'][test_group] = accuracy
    performance_classification['MCC'][test_group] = mcc
    performance_classification['AUC'][test_group] = area_under_curve

# Output the performance metrics for each fold
for metric, values in performance_classification.items():
    for group, value in values.items():
        print(f"{metric} for test group {group}: {value:.4f}")
    
    # Calculate and print the average performance across all folds for each metric
    average_value = np.mean(list(values.values()))
    print(f"Average {metric} across all folds: {average_value:.4f}")
    print('---')  # Separator for clarity


ACC for test group Group 1: 0.7651
ACC for test group Group 2: 0.7399
ACC for test group Group 3: 0.7665
ACC for test group Group 4: 0.7563
ACC for test group Group 5: 0.7690
ACC for test group Group 6: 0.8219
ACC for test group Group 7: 0.7481
ACC for test group Group 8: 0.7455
ACC for test group Group 9: 0.7201
ACC for test group Group 10: 0.7226
Average ACC across all folds: 0.7555
---
MCC for test group Group 1: 0.1945
MCC for test group Group 2: 0.0828
MCC for test group Group 3: 0.0594
MCC for test group Group 4: -0.0042
MCC for test group Group 5: 0.0400
MCC for test group Group 6: 0.1368
MCC for test group Group 7: 0.0558
MCC for test group Group 8: 0.2970
MCC for test group Group 9: 0.0610
MCC for test group Group 10: 0.0464
Average MCC across all folds: 0.0969
---
AUC for test group Group 1: 0.6522
AUC for test group Group 2: 0.7352
AUC for test group Group 3: 0.6695
AUC for test group Group 4: 0.7055
AUC for test group Group 5: 0.6802
AUC for test group Group 6: 0.6996
AUC f

## Random forests + Grid search

In [ ]:
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import GridSearchCV

# Initialize a structure to store the performance of the model on each fold
performance_metrics = {
    'RMSE': {},
    'MSE': {},
    'MAE': {},
    'R2': {}
}

# Define the parameter grid for RandomForestRegressor
param_grid = {
    'n_estimators': [100, 200, 300],  # Number of trees in the forest
    'max_depth': [None, 10, 20, 30],  # Maximum depth of the tree
    # Add more parameters here if needed
}

for test_group in final_group_arrays_hold_out.keys():
    # Prepare the training data: use all groups except the test_group
    train_data = np.concatenate([array for group, array in final_group_arrays_hold_out.items() if group != test_group])
    train_labels = np.concatenate([grouped_dfs_hold_out[group]['DeltaDeltaG'].values for group in grouped_dfs_hold_out if group != test_group])
    
    # Ensure the train_labels is a flat array
    train_labels = train_labels.ravel()
    
    # Prepare the test data
    test_data = final_group_arrays_hold_out[test_group]
    test_labels = grouped_dfs_hold_out[test_group]['DeltaDeltaG'].values
    
    # Initialize the RandomForestRegressor
    rf = RandomForestRegressor(random_state=42)
    
    # Initialize the GridSearchCV
    grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=3, n_jobs=-1, verbose=2, scoring='neg_mean_squared_error')
    
    # Fit the grid search to the data
    grid_search.fit(train_data, train_labels)
    
    # Get the best estimator
    best_rf = grid_search.best_estimator_
    
    # Make predictions on the test set using the best found parameters
    predictions = best_rf.predict(test_data)
    
    # Calculate the performance metrics
    mse = mean_squared_error(test_labels, predictions)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(test_labels, predictions)
    r2 = r2_score(test_labels, predictions)
    
    # Store the performance metrics
    performance_metrics['MSE'][test_group] = mse
    performance_metrics['RMSE'][test_group] = rmse
    performance_metrics['MAE'][test_group] = mae
    performance_metrics['R2'][test_group] = r2

# Output the performance metrics for each fold
for metric, values in performance_metrics.items():
    for group, value in values.items():
        print(f"{metric} for test group {group}: {value:.4f}")
    
    # Calculate and print the average performance across all folds for each metric
    average_value = np.mean(list(values.values()))
    print(f"Average {metric} across all folds: {average_value:.4f}")
    print('---')  # Separator for clarity


Fitting 3 folds for each of 12 candidates, totalling 36 fits
Fitting 3 folds for each of 12 candidates, totalling 36 fits
Fitting 3 folds for each of 12 candidates, totalling 36 fits
Fitting 3 folds for each of 12 candidates, totalling 36 fits
Fitting 3 folds for each of 12 candidates, totalling 36 fits
Fitting 3 folds for each of 12 candidates, totalling 36 fits
Fitting 3 folds for each of 12 candidates, totalling 36 fits
Fitting 3 folds for each of 12 candidates, totalling 36 fits
Fitting 3 folds for each of 12 candidates, totalling 36 fits
Fitting 3 folds for each of 12 candidates, totalling 36 fits
RMSE for test group Group 1: 2.5307
RMSE for test group Group 2: 1.4488
RMSE for test group Group 3: 1.3645
RMSE for test group Group 4: 0.7682
RMSE for test group Group 5: 1.1277
RMSE for test group Group 6: 1.2548
RMSE for test group Group 7: 1.4474
RMSE for test group Group 8: 1.5588
RMSE for test group Group 9: 1.0992
RMSE for test group Group 10: 1.1813
Average RMSE across all folds

In [ ]:
# After fitting GridSearchCV
print("Best parameters found: ", grid_search.best_params_)
print("Best score found: ", grid_search.best_score_)


Best parameters found:  {'max_depth': 10, 'n_estimators': 200}
Best score found:  -2.7811031445772953


## Random forest with vectors of 64 elements

In [ ]:
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Initialize a structure to store the performance of the model on each fold
performance_metrics = {
    'MSE': {},
    'MAE': {},
    'R2': {}
}

for test_group in final_group_arrays_hold_out.keys():
    # Prepare the training data: use all groups except the test_group
    train_data = np.concatenate([array for group, array in final_group_arrays_hold_out.items() if group != test_group])
    train_labels = np.concatenate([grouped_dfs_hold_out[group]['DeltaDeltaG'].values for group in grouped_dfs_hold_out if group != test_group])
    
    # Ensure the train_labels is a flat array
    train_labels = train_labels.ravel()
    
    # Prepare the test data
    test_data = final_group_arrays_hold_out[test_group]
    test_labels = grouped_dfs_hold_out[test_group]['DeltaDeltaG'].values
    
    # Initialize the model with the best parameters
    model = RandomForestRegressor(max_depth=10, n_estimators=200, random_state=42)
    
    # Train the model
    model.fit(train_data, train_labels)
    
    # Make predictions on the test set
    predictions = model.predict(test_data)
    
    # Calculate the performance metrics
    mse = mean_squared_error(test_labels, predictions)
    mae = mean_absolute_error(test_labels, predictions)
    r2 = r2_score(test_labels, predictions)
    
    # Store the performance metrics
    performance_metrics['MSE'][test_group] = mse
    performance_metrics['MAE'][test_group] = mae
    performance_metrics['R2'][test_group] = r2

# Output the performance metrics for each fold
for metric, values in performance_metrics.items():
    for group, value in values.items():
        print(f"{metric} for test group {group}: {value:.4f}")
    
    # Calculate and print the average performance across all folds for each metric
    average_value = np.mean(list(values.values()))
    print(f"Average {metric} across all folds: {average_value:.4f}")
    print('---')  # Separator for clarity


MSE for test group Group 1: 6.4289
MSE for test group Group 2: 2.0854
MSE for test group Group 3: 1.7333
MSE for test group Group 4: 0.5764
MSE for test group Group 5: 1.3608
MSE for test group Group 6: 1.5616
MSE for test group Group 7: 2.0943
MSE for test group Group 8: 2.3759
MSE for test group Group 9: 1.2266
MSE for test group Group 10: 1.4166
Average MSE across all folds: 2.0860
---
MAE for test group Group 1: 1.7452
MAE for test group Group 2: 1.0087
MAE for test group Group 3: 0.9058
MAE for test group Group 4: 0.5505
MAE for test group Group 5: 0.8028
MAE for test group Group 6: 0.8563
MAE for test group Group 7: 1.0593
MAE for test group Group 8: 1.0740
MAE for test group Group 9: 0.8262
MAE for test group Group 10: 0.8628
Average MAE across all folds: 0.9692
---
R2 for test group Group 1: 0.0137
R2 for test group Group 2: 0.2109
R2 for test group Group 3: -0.0626
R2 for test group Group 4: 0.2791
R2 for test group Group 5: 0.0976
R2 for test group Group 6: 0.2103
R2 for test

## Linear Regression 

In [ ]:
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Initialize a structure to store the performance of the model on each fold
performance_metrics = {
    'MSE': {},
    'MAE': {},
    'R2': {}
}

for test_group in final_group_arrays_hold_out.keys():
    # Prepare the training data: use all groups except the test_group
    train_data = np.concatenate([array for group, array in final_group_arrays_hold_out.items() if group != test_group])
    train_labels = np.concatenate([grouped_dfs_hold_out[group]['DeltaDeltaG'].values for group in grouped_dfs_hold_out if group != test_group])
    
    # Ensure the train_labels is a flat array
    train_labels = train_labels.ravel()
    
    # Prepare the test data
    test_data = final_group_arrays_hold_out[test_group]
    test_labels = grouped_dfs_hold_out[test_group]['DeltaDeltaG'].values
    
    # Initialize the model
    model = LinearRegression()
    
    # Train the model
    model.fit(train_data, train_labels)
    
    # Make predictions on the test set
    predictions = model.predict(test_data)
    
    # Calculate the performance metrics
    mse = mean_squared_error(test_labels, predictions)
    mae = mean_absolute_error(test_labels, predictions)
    r2 = r2_score(test_labels, predictions)
    
    # Store the performance metrics
    performance_metrics['MSE'][test_group] = mse
    performance_metrics['MAE'][test_group] = mae
    performance_metrics['R2'][test_group] = r2

# Output the performance metrics for each fold
for metric, values in performance_metrics.items():
    for group, value in values.items():
        print(f"{metric} for test group {group}: {value:.4f}")
    
    # Calculate and print the average performance across all folds for each metric
    average_value = np.mean(list(values.values()))
    print(f"Average {metric} across all folds: {average_value:.4f}")
    print('---')  # Separator for clarity


MSE for test group Group 1: 6.4075
MSE for test group Group 2: 2.2924
MSE for test group Group 3: 1.7011
MSE for test group Group 4: 0.7171
MSE for test group Group 5: 1.3523
MSE for test group Group 6: 1.5475
MSE for test group Group 7: 2.2360
MSE for test group Group 8: 2.7031
MSE for test group Group 9: 1.3641
MSE for test group Group 10: 3.3606
Average MSE across all folds: 2.3682
---
MAE for test group Group 1: 1.7794
MAE for test group Group 2: 1.0838
MAE for test group Group 3: 0.9394
MAE for test group Group 4: 0.6527
MAE for test group Group 5: 0.8470
MAE for test group Group 6: 0.8780
MAE for test group Group 7: 1.1091
MAE for test group Group 8: 1.1492
MAE for test group Group 9: 0.8888
MAE for test group Group 10: 1.0476
Average MAE across all folds: 1.0375
---
R2 for test group Group 1: 0.0170
R2 for test group Group 2: 0.1326
R2 for test group Group 3: -0.0429
R2 for test group Group 4: 0.1031
R2 for test group Group 5: 0.1032
R2 for test group Group 6: 0.2174
R2 for test

## Linear Regression with 64 elements

In [ ]:
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Initialize a structure to store the performance of the model on each fold
performance_metrics = {
    'MSE': {},
    'MAE': {},
    'R2': {}
}

for test_group in final_group_arrays_hold_out.keys():
    # Prepare the training data: use all groups except the test_group
    train_data = np.concatenate([array for group, array in final_group_arrays_hold_out.items() if group != test_group])
    train_labels = np.concatenate([grouped_dfs_hold_out[group]['DeltaDeltaG'].values for group in grouped_dfs_hold_out if group != test_group])
    
    # Ensure the train_labels is a flat array
    train_labels = train_labels.ravel()
    
    # Prepare the test data
    test_data = final_group_arrays_hold_out[test_group]
    test_labels = grouped_dfs_hold_out[test_group]['DeltaDeltaG'].values
    
    # Initialize the model
    model = LinearRegression()
    
    # Train the model
    model.fit(train_data, train_labels)
    
    # Make predictions on the test set
    predictions = model.predict(test_data)
    
    # Calculate the performance metrics
    mse = mean_squared_error(test_labels, predictions)
    mae = mean_absolute_error(test_labels, predictions)
    r2 = r2_score(test_labels, predictions)
    
    # Store the performance metrics
    performance_metrics['MSE'][test_group] = mse
    performance_metrics['MAE'][test_group] = mae
    performance_metrics['R2'][test_group] = r2

# Output the performance metrics for each fold
for metric, values in performance_metrics.items():
    for group, value in values.items():
        print(f"{metric} for test group {group}: {value:.4f}")
    
    # Calculate and print the average performance across all folds for each metric
    average_value = np.mean(list(values.values()))
    print(f"Average {metric} across all folds: {average_value:.4f}")
    print('---')  # Separator for clarity

MSE for test group Group 1: 6.3590
MSE for test group Group 2: 2.2785
MSE for test group Group 3: 1.7178
MSE for test group Group 4: 0.7341
MSE for test group Group 5: 1.4156
MSE for test group Group 6: 1.5794
MSE for test group Group 7: 2.2119
MSE for test group Group 8: 3.0366
MSE for test group Group 9: 1.4348
MSE for test group Group 10: 3.3819
Average MSE across all folds: 2.4150
---
MAE for test group Group 1: 1.7721
MAE for test group Group 2: 1.0979
MAE for test group Group 3: 0.9299
MAE for test group Group 4: 0.6617
MAE for test group Group 5: 0.8686
MAE for test group Group 6: 0.8888
MAE for test group Group 7: 1.1163
MAE for test group Group 8: 1.2458
MAE for test group Group 9: 0.9146
MAE for test group Group 10: 1.0588
Average MAE across all folds: 1.0555
---
R2 for test group Group 1: 0.0244
R2 for test group Group 2: 0.1378
R2 for test group Group 3: -0.0531
R2 for test group Group 4: 0.0818
R2 for test group Group 5: 0.0612
R2 for test group Group 6: 0.2013
R2 for test

## Gradient boost regressor 

In [ ]:
import numpy as np
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Initialize a structure to store the performance of the model on each fold
performance_metrics = {
    'MSE': {},
    'MAE': {},
    'R2': {}
}

for test_group in final_group_arrays_hold_out.keys():
    # Prepare the training data: use all groups except the test_group
    train_data = np.concatenate([array for group, array in final_group_arrays_hold_out.items() if group != test_group])
    train_labels = np.concatenate([grouped_dfs_hold_out[group]['DeltaDeltaG'].values for group in grouped_dfs_hold_out if group != test_group])
    
    # Ensure the train_labels is a flat array
    train_labels = train_labels.ravel()
    
    # Prepare the test data
    test_data = final_group_arrays_hold_out[test_group]
    test_labels = grouped_dfs_hold_out[test_group]['DeltaDeltaG'].values
    
    # Initialize the model
    model = GradientBoostingRegressor(random_state=42)
    
    # Train the model
    model.fit(train_data, train_labels)
    
    # Make predictions on the test set
    predictions = model.predict(test_data)
    
    # Calculate the performance metrics
    mse = mean_squared_error(test_labels, predictions)
    mae = mean_absolute_error(test_labels, predictions)
    r2 = r2_score(test_labels, predictions)
    
    # Store the performance metrics
    performance_metrics['MSE'][test_group] = mse
    performance_metrics['MAE'][test_group] = mae
    performance_metrics['R2'][test_group] = r2

# Output the performance metrics for each fold
for metric, values in performance_metrics.items():
    for group, value in values.items():
        print(f"{metric} for test group {group}: {value:.4f}")
    
    # Calculate and print the average performance across all folds for each metric
    average_value = np.mean(list(values.values()))
    print(f"Average {metric} across all folds: {average_value:.4f}")
    print('---')  # Separator for clarity


MSE for test group Group 1: 5.9266
MSE for test group Group 2: 1.9704
MSE for test group Group 3: 1.9078
MSE for test group Group 4: 0.5987
MSE for test group Group 5: 1.1880
MSE for test group Group 6: 1.6233
MSE for test group Group 7: 1.9893
MSE for test group Group 8: 2.4301
MSE for test group Group 9: 1.2133
MSE for test group Group 10: 1.4730
Average MSE across all folds: 2.0320
---
MAE for test group Group 1: 1.6754
MAE for test group Group 2: 0.9808
MAE for test group Group 3: 0.9608
MAE for test group Group 4: 0.5751
MAE for test group Group 5: 0.7636
MAE for test group Group 6: 0.8716
MAE for test group Group 7: 1.0417
MAE for test group Group 8: 1.1014
MAE for test group Group 9: 0.8400
MAE for test group Group 10: 0.8923
Average MAE across all folds: 0.9703
---
R2 for test group Group 1: 0.0908
R2 for test group Group 2: 0.2544
R2 for test group Group 3: -0.1696
R2 for test group Group 4: 0.2512
R2 for test group Group 5: 0.2122
R2 for test group Group 6: 0.1791
R2 for test

## Best parameters gradient boost

In [ ]:
import numpy as np
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Initialize a structure to store the performance of the model on each fold
performance_metrics = {
    'MSE': {},
    'MAE': {},
    'R2': {}
}

for test_group in final_group_arrays_hold_out.keys():
    # Prepare the training data: use all groups except the test_group
    train_data = np.concatenate([array for group, array in final_group_arrays_hold_out.items() if group != test_group])
    train_labels = np.concatenate([grouped_dfs_hold_out[group]['DeltaDeltaG'].values for group in grouped_dfs_hold_out if group != test_group])
    
    # Ensure the train_labels is a flat array
    train_labels = train_labels.ravel()
    
    # Prepare the test data
    test_data = final_group_arrays_hold_out[test_group]
    test_labels = grouped_dfs_hold_out[test_group]['DeltaDeltaG'].values
    
    # Initialize the model with the best parameters
    model = GradientBoostingRegressor(
        learning_rate=0.01,
        max_depth=5,
        n_estimators=300,
        random_state=42
    )
    
    # Train the model
    model.fit(train_data, train_labels)
    
    # Make predictions on the test set
    predictions = model.predict(test_data)
    
    # Calculate the performance metrics
    mse = mean_squared_error(test_labels, predictions)
    mae = mean_absolute_error(test_labels, predictions)
    r2 = r2_score(test_labels, predictions)
    
    # Store the performance metrics
    performance_metrics['MSE'][test_group] = mse
    performance_metrics['MAE'][test_group] = mae
    performance_metrics['R2'][test_group] = r2

# Output the performance metrics for each fold
for metric, values in performance_metrics.items():
    for group, value in values.items():
        print(f"{metric} for test group {group}: {value:.4f}")
    
    # Calculate and print the average performance across all folds for each metric
    average_value = np.mean(list(values.values()))
    print(f"Average {metric} across all folds: {average_value:.4f}")
    print('---')  # Separator for clarity


MSE for test group Group 1: 6.0875
MSE for test group Group 2: 2.0792
MSE for test group Group 3: 1.7966
MSE for test group Group 4: 0.6152
MSE for test group Group 5: 1.2444
MSE for test group Group 6: 1.6146
MSE for test group Group 7: 2.0763
MSE for test group Group 8: 2.4188
MSE for test group Group 9: 1.1884
MSE for test group Group 10: 1.3970
Average MSE across all folds: 2.0518
---
MAE for test group Group 1: 1.6876
MAE for test group Group 2: 0.9998
MAE for test group Group 3: 0.9254
MAE for test group Group 4: 0.5849
MAE for test group Group 5: 0.7710
MAE for test group Group 6: 0.8667
MAE for test group Group 7: 1.0767
MAE for test group Group 8: 1.0792
MAE for test group Group 9: 0.8384
MAE for test group Group 10: 0.8717
Average MAE across all folds: 0.9701
---
R2 for test group Group 1: 0.0661
R2 for test group Group 2: 0.2133
R2 for test group Group 3: -0.1014
R2 for test group Group 4: 0.2306
R2 for test group Group 5: 0.1747
R2 for test group Group 6: 0.1835
R2 for test

## Gradient boost + Grid search 

In [ ]:
import numpy as np
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import GridSearchCV

# Define the parameter grid to search
param_grid = {
    'max_depth': [3, 5, 10],
    'n_estimators': [100, 200, 300],
    'learning_rate': [0.01, 0.1, 0.2]
}

# Initialize a structure to store the performance of the model on each fold
performance_metrics = {
    'MSE': {},
    'MAE': {},
    'R2': {}
}

for test_group in final_group_arrays_hold_out.keys():
    # Prepare the training data: use all groups except the test_group
    train_data = np.concatenate([array for group, array in final_group_arrays_hold_out.items() if group != test_group])
    train_labels = np.concatenate([grouped_dfs_hold_out[group]['DeltaDeltaG'].values for group in grouped_dfs_hold_out if group != test_group])
    
    train_labels = train_labels.ravel()  # Ensure the train_labels is a flat array
    
    # Prepare the test data
    test_data = final_group_arrays_hold_out[test_group]
    test_labels = grouped_dfs_hold_out[test_group]['DeltaDeltaG'].values
    
    # Initialize the model
    model = GradientBoostingRegressor(random_state=42)
    
    # Initialize GridSearchCV
    grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=5, n_jobs=-1, scoring='neg_mean_squared_error', verbose=1)
    
    # Fit the grid search to the data
    grid_search.fit(train_data, train_labels)
    
    # Get the best estimator
    best_model = grid_search.best_estimator_
    
    # Make predictions on the test set using the best found parameters
    predictions = best_model.predict(test_data)
    
    # Calculate the performance metrics
    mse = mean_squared_error(test_labels, predictions)
    mae = mean_absolute_error(test_labels, predictions)
    r2 = r2_score(test_labels, predictions)
    
    # Store the performance metrics
    performance_metrics['MSE'][test_group] = mse
    performance_metrics['MAE'][test_group] = mae
    performance_metrics['R2'][test_group] = r2

# Output the performance metrics for each fold
for metric, values in performance_metrics.items():
    for group, value in values.items():
        print(f"{metric} for test group {group}: {value:.4f}")
    
    # Calculate and print the average performance across all folds for each metric
    average_value = np.mean(list(values.values()))
    print(f"Average {metric} across all folds: {average_value:.4f}")
    print('---')  # Separator for clarity


Fitting 5 folds for each of 27 candidates, totalling 135 fits
Fitting 5 folds for each of 27 candidates, totalling 135 fits
Fitting 5 folds for each of 27 candidates, totalling 135 fits
Fitting 5 folds for each of 27 candidates, totalling 135 fits
Fitting 5 folds for each of 27 candidates, totalling 135 fits
Fitting 5 folds for each of 27 candidates, totalling 135 fits
Fitting 5 folds for each of 27 candidates, totalling 135 fits
Fitting 5 folds for each of 27 candidates, totalling 135 fits
Fitting 5 folds for each of 27 candidates, totalling 135 fits
Fitting 5 folds for each of 27 candidates, totalling 135 fits
MSE for test group Group 1: 5.9266
MSE for test group Group 2: 2.0792
MSE for test group Group 3: 1.8700
MSE for test group Group 4: 0.6152
MSE for test group Group 5: 1.1880
MSE for test group Group 6: 1.6233
MSE for test group Group 7: 1.9893
MSE for test group Group 8: 2.5734
MSE for test group Group 9: 1.2443
MSE for test group Group 10: 1.3970
Average MSE across all folds:

In [ ]:
print("Best parameters found: ", grid_search.best_params_)
print("Best score found: ", grid_search.best_score_)


Best parameters found:  {'learning_rate': 0.01, 'max_depth': 5, 'n_estimators': 300}
Best score found:  -2.6662961994782988


## Gradient boost with 64 elements

In [36]:
import numpy as np
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Initialize a structure to store the performance of the model on each fold
performance_metrics = {
    'MSE': {},
    'MAE': {},
    'R2': {}
}

for test_group in final_group_arrays_hold_out.keys():
    # Prepare the training data: use all groups except the test_group
    train_data = np.concatenate([array for group, array in final_group_arrays_hold_out.items() if group != test_group])
    train_labels = np.concatenate([grouped_dfs_hold_out[group]['DeltaDeltaG'].values for group in grouped_dfs_hold_out if group != test_group])
    
    # Ensure the train_labels is a flat array
    train_labels = train_labels.ravel()
    
    # Prepare the test data
    test_data = final_group_arrays_hold_out[test_group]
    test_labels = grouped_dfs_hold_out[test_group]['DeltaDeltaG'].values
    
    # Initialize the model with the best parameters
    model = GradientBoostingRegressor(
        learning_rate=0.01,
        max_depth=5,
        n_estimators=300,
        random_state=42
    )
    
    # Train the model
    model.fit(train_data, train_labels)
    
    # Make predictions on the test set
    predictions = model.predict(test_data)
    
    # Calculate the performance metrics
    mse = mean_squared_error(test_labels, predictions)
    mae = mean_absolute_error(test_labels, predictions)
    r2 = r2_score(test_labels, predictions)
    
    # Store the performance metrics
    performance_metrics['MSE'][test_group] = mse
    performance_metrics['MAE'][test_group] = mae
    performance_metrics['R2'][test_group] = r2

# Output the performance metrics for each fold
for metric, values in performance_metrics.items():
    for group, value in values.items():
        print(f"{metric} for test group {group}: {value:.4f}")
    
    # Calculate and print the average performance across all folds for each metric
    average_value = np.mean(list(values.values()))
    print(f"Average {metric} across all folds: {average_value:.4f}")
    print('---')  # Separator for clarity

MSE for test group Group 1: 6.3236
MSE for test group Group 2: 2.1452
MSE for test group Group 3: 1.6881
MSE for test group Group 4: 0.6472
MSE for test group Group 5: 1.3119
MSE for test group Group 6: 1.4782
MSE for test group Group 7: 2.0456
MSE for test group Group 8: 2.3248
MSE for test group Group 9: 1.2053
MSE for test group Group 10: 1.4482
Average MSE across all folds: 2.0618
---
MAE for test group Group 1: 1.7114
MAE for test group Group 2: 1.0141
MAE for test group Group 3: 0.8875
MAE for test group Group 4: 0.5974
MAE for test group Group 5: 0.7969
MAE for test group Group 6: 0.8491
MAE for test group Group 7: 1.0601
MAE for test group Group 8: 1.0571
MAE for test group Group 9: 0.8365
MAE for test group Group 10: 0.8741
Average MAE across all folds: 0.9684
---
R2 for test group Group 1: 0.0299
R2 for test group Group 2: 0.1883
R2 for test group Group 3: -0.0349
R2 for test group Group 4: 0.1906
R2 for test group Group 5: 0.1300
R2 for test group Group 6: 0.2525
R2 for test

## Support vector Regression

In [ ]:
import numpy as np
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Initialize a structure to store the performance of the model on each fold
performance_metrics = {
    'MSE': {},
    'MAE': {},
    'R2': {}
}

for test_group in final_group_arrays_hold_out.keys():
    # Prepare the training data: use all groups except the test_group
    train_data = np.concatenate([array for group, array in final_group_arrays_hold_out.items() if group != test_group])
    train_labels = np.concatenate([grouped_dfs_hold_out[group]['DeltaDeltaG'].values for group in grouped_dfs_hold_out if group != test_group])
    
    # Ensure the train_labels is a flat array
    train_labels = train_labels.ravel()
    
    # Prepare the test data
    test_data = final_group_arrays_hold_out[test_group]
    test_labels = grouped_dfs_hold_out[test_group]['DeltaDeltaG'].values
    
    # Initialize the model with an RBF kernel
    model = SVR(kernel='rbf')
    
    # Train the model
    model.fit(train_data, train_labels)
    
    # Make predictions on the test set
    predictions = model.predict(test_data)
    
    # Calculate the performance metrics
    mse = mean_squared_error(test_labels, predictions)
    mae = mean_absolute_error(test_labels, predictions)
    r2 = r2_score(test_labels, predictions)
    
    # Store the performance metrics
    performance_metrics['MSE'][test_group] = mse
    performance_metrics['MAE'][test_group] = mae
    performance_metrics['R2'][test_group] = r2

# Output the performance metrics for each fold
for metric, values in performance_metrics.items():
    for group, value in values.items():
        print(f"{metric} for test group {group}: {value:.4f}")
    
    # Calculate and print the average performance across all folds for each metric
    average_value = np.mean(list(values.values()))
    print(f"Average {metric} across all folds: {average_value:.4f}")
    print('---')  # Separator for clarity


MSE for test group Group 1: 6.6538
MSE for test group Group 2: 2.4905
MSE for test group Group 3: 1.5529
MSE for test group Group 4: 0.5671
MSE for test group Group 5: 1.2073
MSE for test group Group 6: 1.5281
MSE for test group Group 7: 2.0867
MSE for test group Group 8: 2.4372
MSE for test group Group 9: 1.2197
MSE for test group Group 10: 1.4972
Average MSE across all folds: 2.1240
---
MAE for test group Group 1: 1.7783
MAE for test group Group 2: 1.0849
MAE for test group Group 3: 0.8417
MAE for test group Group 4: 0.5398
MAE for test group Group 5: 0.7465
MAE for test group Group 6: 0.8411
MAE for test group Group 7: 1.0688
MAE for test group Group 8: 1.0687
MAE for test group Group 9: 0.8313
MAE for test group Group 10: 0.9033
Average MAE across all folds: 0.9704
---
R2 for test group Group 1: -0.0208
R2 for test group Group 2: 0.0576
R2 for test group Group 3: 0.0480
R2 for test group Group 4: 0.2908
R2 for test group Group 5: 0.1993
R2 for test group Group 6: 0.2273
R2 for test

## Best parameters support vector regression

In [ ]:
import numpy as np
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Initialize a structure to store the performance of the model on each fold
performance_metrics = {
    'MSE': {},
    'MAE': {},
    'R2': {}
}

for test_group in final_group_arrays_hold_out.keys():
    # Prepare the training data: use all groups except the test_group
    train_data = np.concatenate([array for group, array in final_group_arrays_hold_out.items() if group != test_group])
    train_labels = np.concatenate([grouped_dfs_hold_out[group]['DeltaDeltaG'].values for group in grouped_dfs_hold_out if group != test_group])
    
    # Ensure the train_labels is a flat array
    train_labels = train_labels.ravel()
    
    # Prepare the test data
    test_data = final_group_arrays_hold_out[test_group]
    test_labels = grouped_dfs_hold_out[test_group]['DeltaDeltaG'].values
    
    # Initialize the model with the best parameters
    model = SVR(kernel='rbf', C=1, gamma='scale')
    
    # Train the model
    model.fit(train_data, train_labels)
    
    # Make predictions on the test set
    predictions = model.predict(test_data)
    
    # Calculate the performance metrics
    mse = mean_squared_error(test_labels, predictions)
    mae = mean_absolute_error(test_labels, predictions)
    r2 = r2_score(test_labels, predictions)
    
    # Store the performance metrics
    performance_metrics['MSE'][test_group] = mse
    performance_metrics['MAE'][test_group] = mae
    performance_metrics['R2'][test_group] = r2

# Output the performance metrics for each fold
for metric, values in performance_metrics.items():
    for group, value in values.items():
        print(f"{metric} for test group {group}: {value:.4f}")
    
    # Calculate and print the average performance across all folds for each metric
    average_value = np.mean(list(values.values()))
    print(f"Average {metric} across all folds: {average_value:.4f}")
    print('---')  # Separator for clarity


MSE for test group Group 1: 6.6538
MSE for test group Group 2: 2.4905
MSE for test group Group 3: 1.5529
MSE for test group Group 4: 0.5671
MSE for test group Group 5: 1.2073
MSE for test group Group 6: 1.5281
MSE for test group Group 7: 2.0867
MSE for test group Group 8: 2.4372
MSE for test group Group 9: 1.2197
MSE for test group Group 10: 1.4972
Average MSE across all folds: 2.1240
---
MAE for test group Group 1: 1.7783
MAE for test group Group 2: 1.0849
MAE for test group Group 3: 0.8417
MAE for test group Group 4: 0.5398
MAE for test group Group 5: 0.7465
MAE for test group Group 6: 0.8411
MAE for test group Group 7: 1.0688
MAE for test group Group 8: 1.0687
MAE for test group Group 9: 0.8313
MAE for test group Group 10: 0.9033
Average MAE across all folds: 0.9704
---
R2 for test group Group 1: -0.0208
R2 for test group Group 2: 0.0576
R2 for test group Group 3: 0.0480
R2 for test group Group 4: 0.2908
R2 for test group Group 5: 0.1993
R2 for test group Group 6: 0.2273
R2 for test

## Support Vector Regression + Grid Search 

In [ ]:
import numpy as np
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import GridSearchCV

# Define the parameter grid to search
param_grid = {
    'kernel': ['rbf', 'sigmoid'],
    'C': [0.1, 1, 10],
    'gamma': ['scale', 'auto']
}

# Initialize a structure to store the performance of the model on each fold
performance_metrics = {
    'MSE': {},
    'MAE': {},
    'R2': {}
}

for test_group in final_group_arrays_hold_out.keys():
    # Prepare the training data: use all groups except the test_group
    train_data = np.concatenate([array for group, array in final_group_arrays_hold_out.items() if group != test_group])
    train_labels = np.concatenate([grouped_dfs_hold_out[group]['DeltaDeltaG'].values for group in grouped_dfs_hold_out if group != test_group])
    
    train_labels = train_labels.ravel()  # Ensure the train_labels is a flat array
    
    # Prepare the test data
    test_data = final_group_arrays_hold_out[test_group]
    test_labels = grouped_dfs_hold_out[test_group]['DeltaDeltaG'].values
    
    # Initialize the model
    model = SVR()
    
    # Initialize GridSearchCV
    grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=5, n_jobs=-1, scoring='neg_mean_squared_error', verbose=1)
    
    # Fit the grid search to the data
    grid_search.fit(train_data, train_labels)
    
    # Get the best estimator
    best_model = grid_search.best_estimator_
    
    # Make predictions on the test set using the best found parameters
    predictions = best_model.predict(test_data)
    
    # Calculate the performance metrics
    mse = mean_squared_error(test_labels, predictions)
    mae = mean_absolute_error(test_labels, predictions)
    r2 = r2_score(test_labels, predictions)
    
    # Store the performance metrics
    performance_metrics['MSE'][test_group] = mse
    performance_metrics['MAE'][test_group] = mae
    performance_metrics['R2'][test_group] = r2

# Output the performance metrics for each fold
for metric, values in performance_metrics.items():
    for group, value in values.items():
        print(f"{metric} for test group {group}: {value:.4f}")
    
    # Calculate and print the average performance across all folds for each metric
    average_value = np.mean(list(values.values()))
    print(f"Average {metric} across all folds: {average_value:.4f}")
    print('---')  # Separator for clarity


Fitting 5 folds for each of 12 candidates, totalling 60 fits
Fitting 5 folds for each of 12 candidates, totalling 60 fits
Fitting 5 folds for each of 12 candidates, totalling 60 fits
Fitting 5 folds for each of 12 candidates, totalling 60 fits
Fitting 5 folds for each of 12 candidates, totalling 60 fits
Fitting 5 folds for each of 12 candidates, totalling 60 fits
Fitting 5 folds for each of 12 candidates, totalling 60 fits
Fitting 5 folds for each of 12 candidates, totalling 60 fits
Fitting 5 folds for each of 12 candidates, totalling 60 fits
Fitting 5 folds for each of 12 candidates, totalling 60 fits
MSE for test group Group 1: 6.6868
MSE for test group Group 2: 3.0581
MSE for test group Group 3: 1.5989
MSE for test group Group 4: 0.5671
MSE for test group Group 5: 1.2073
MSE for test group Group 6: 1.5281
MSE for test group Group 7: 2.0867
MSE for test group Group 8: 2.4372
MSE for test group Group 9: 1.2197
MSE for test group Group 10: 1.4972
Average MSE across all folds: 2.1887
--

In [ ]:
print("Best parameters found: ", grid_search.best_params_)
print("Best score found: ", grid_search.best_score_)

Best parameters found:  {'C': 1, 'gamma': 'scale', 'kernel': 'rbf'}
Best score found:  -2.700492086155


## Support Vector Regression with 64 elements 

In [37]:
import numpy as np
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Initialize a structure to store the performance of the model on each fold
performance_metrics = {
    'MSE': {},
    'MAE': {},
    'R2': {}
}

for test_group in final_group_arrays_hold_out.keys():
    # Prepare the training data: use all groups except the test_group
    train_data = np.concatenate([array for group, array in final_group_arrays_hold_out.items() if group != test_group])
    train_labels = np.concatenate([grouped_dfs_hold_out[group]['DeltaDeltaG'].values for group in grouped_dfs_hold_out if group != test_group])
    
    # Ensure the train_labels is a flat array
    train_labels = train_labels.ravel()
    
    # Prepare the test data
    test_data = final_group_arrays_hold_out[test_group]
    test_labels = grouped_dfs_hold_out[test_group]['DeltaDeltaG'].values
    
    # Initialize the model with the best parameters
    model = GradientBoostingRegressor(
        learning_rate=0.01,
        max_depth=5,
        n_estimators=300,
        random_state=42
    )
    
    # Train the model
    model.fit(train_data, train_labels)
    
    # Make predictions on the test set
    predictions = model.predict(test_data)
    
    # Calculate the performance metrics
    mse = mean_squared_error(test_labels, predictions)
    mae = mean_absolute_error(test_labels, predictions)
    r2 = r2_score(test_labels, predictions)
    
    # Store the performance metrics
    performance_metrics['MSE'][test_group] = mse
    performance_metrics['MAE'][test_group] = mae
    performance_metrics['R2'][test_group] = r2

# Output the performance metrics for each fold
for metric, values in performance_metrics.items():
    for group, value in values.items():
        print(f"{metric} for test group {group}: {value:.4f}")
    
    # Calculate and print the average performance across all folds for each metric
    average_value = np.mean(list(values.values()))
    print(f"Average {metric} across all folds: {average_value:.4f}")
    print('---')  # Separator for clarity

MSE for test group Group 1: 6.3236
MSE for test group Group 2: 2.1452
MSE for test group Group 3: 1.6881
MSE for test group Group 4: 0.6472
MSE for test group Group 5: 1.3119
MSE for test group Group 6: 1.4782
MSE for test group Group 7: 2.0456
MSE for test group Group 8: 2.3248
MSE for test group Group 9: 1.2053
MSE for test group Group 10: 1.4482
Average MSE across all folds: 2.0618
---
MAE for test group Group 1: 1.7114
MAE for test group Group 2: 1.0141
MAE for test group Group 3: 0.8875
MAE for test group Group 4: 0.5974
MAE for test group Group 5: 0.7969
MAE for test group Group 6: 0.8491
MAE for test group Group 7: 1.0601
MAE for test group Group 8: 1.0571
MAE for test group Group 9: 0.8365
MAE for test group Group 10: 0.8741
Average MAE across all folds: 0.9684
---
R2 for test group Group 1: 0.0299
R2 for test group Group 2: 0.1883
R2 for test group Group 3: -0.0349
R2 for test group Group 4: 0.1906
R2 for test group Group 5: 0.1300
R2 for test group Group 6: 0.2525
R2 for test

## SVM with classification
 

In [25]:
import numpy as np
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, matthews_corrcoef, roc_auc_score

# Initialize a structure to store the performance of the model on each fold
performance_classification = {
    'ACC': {},
    'MCC': {},
    'AUC': {}
}

for test_group in final_group_arrays_hold_out.keys():
    # Prepare the training data: use all groups except the test_group
    train_data = np.concatenate([array for group, array in final_group_arrays_hold_out.items() if group != test_group])
    train_labels = np.concatenate([grouped_dfs_hold_out[group]['DeltaDeltaG'].values for group in grouped_dfs_hold_out if group != test_group])
    
    # Ensure the train_labels is a flat array and transform to -1 and +1
    train_labels = train_labels.ravel()
    train_labels = np.sign(train_labels)
    train_labels = 2 * (train_labels > 0) - 1  # Transform in +1 and -1
    
    # Prepare the test data and transform labels to -1 and +1
    test_data = final_group_arrays_hold_out[test_group]
    test_labels = np.sign(grouped_dfs_hold_out[test_group]['DeltaDeltaG'].values)
    test_labels = 2 * (test_labels > 0) - 1
    
    # Initialize the model with an RBF kernel
    model = SVC(kernel='rbf', probability=True)  # Set probability=True to enable predict_proba method
    
    # Train the model
    model.fit(train_data, train_labels)
    
    # Make predictions on the test set
    predictions = model.predict(test_data)
    predictions_probability = model.predict_proba(test_data)[:, 1]  # Get probabilities for the positive class

    # Calculate the performance metrics
    accuracy = accuracy_score(test_labels, predictions)
    mcc = matthews_corrcoef(test_labels, predictions)
    area_under_curve = roc_auc_score(test_labels, predictions_probability)  # Compute AUC using roc_auc_score
    
    # Store the performance metrics
    performance_classification['ACC'][test_group] = accuracy
    performance_classification['MCC'][test_group] = mcc
    performance_classification['AUC'][test_group] = area_under_curve

# Output the performance metrics for each fold
for metric, values in performance_classification.items():
    for group, value in values.items():
        print(f"{metric} for test group {group}: {value:.4f}")
    
    # Calculate and print the average performance across all folds for each metric
    average_value = np.mean(list(values.values()))
    print(f"Average {metric} across all folds: {average_value:.4f}")
    print('---')  # Separator for clarity


ACC for test group Group 1: 0.7515
ACC for test group Group 2: 0.7371
ACC for test group Group 3: 0.7665
ACC for test group Group 4: 0.7766
ACC for test group Group 5: 0.7843
ACC for test group Group 6: 0.8193
ACC for test group Group 7: 0.7481
ACC for test group Group 8: 0.7048
ACC for test group Group 9: 0.7176
ACC for test group Group 10: 0.7277
Average ACC across all folds: 0.7534
---
MCC for test group Group 1: 0.0257
MCC for test group Group 2: 0.0285
MCC for test group Group 3: 0.0000
MCC for test group Group 4: 0.0474
MCC for test group Group 5: 0.0951
MCC for test group Group 6: 0.0594
MCC for test group Group 7: -0.0291
MCC for test group Group 8: 0.0000
MCC for test group Group 9: -0.0315
MCC for test group Group 10: 0.0827
Average MCC across all folds: 0.0278
---
AUC for test group Group 1: 0.6149
AUC for test group Group 2: 0.5993
AUC for test group Group 3: 0.6440
AUC for test group Group 4: 0.6515
AUC for test group Group 5: 0.5297
AUC for test group Group 6: 0.7038
AUC 

## Clustering and split of the dataset according to hold_out_protein

In [6]:
import numpy as np
from sklearn.cluster import AgglomerativeClustering

# Assuming skempi_data_filtered is already a pandas DataFrame

# Process the 'Hold_out_proteins' by splitting and converting to sets
skempi_data_filtered['Hold_out_protein_sets'] = skempi_data_filtered['Hold_out_proteins'].apply(lambda x: set(x.split(',')))

# Initialize the distance matrix
num_entries = len(skempi_data_filtered)
distance_matrix = np.ones((num_entries, num_entries))  # initialize with distance 1

# Populate the distance matrix
for i in range(num_entries):
    for j in range(i+1, num_entries):  # no need to compare with itself and already compared pairs
        if len(skempi_data_filtered['Hold_out_protein_sets'].iloc[i].intersection(skempi_data_filtered['Hold_out_protein_sets'].iloc[j])) > 0:
            distance_matrix[i][j] = 0
            distance_matrix[j][i] = 0  # the matrix is symmetric



In [9]:
distance_matrix

array([[1., 0., 0., ..., 1., 1., 1.],
       [0., 1., 0., ..., 1., 1., 1.],
       [0., 0., 1., ..., 1., 1., 1.],
       ...,
       [1., 1., 1., ..., 1., 0., 0.],
       [1., 1., 1., ..., 0., 1., 0.],
       [1., 1., 1., ..., 0., 0., 1.]])

In [11]:
# Save the distance matrix to a binary file in NumPy .npy format
np.save('distance_matrix.npy', distance_matrix)

In [ ]:
# Load the distance matrix from the file
distance_matrix = np.load('distance_matrix.npy')

In [14]:

# Perform Agglomerative Clustering
clustering = AgglomerativeClustering(n_clusters=5, affinity='precomputed', linkage='complete')
labels = clustering.fit_predict(distance_matrix)

# Add the cluster labels to your dataframe
skempi_data_filtered['Cluster_Labels'] = labels

# Display the data with the cluster labels
skempi_data_filtered[['Hold_out_proteins', 'Cluster_Labels']]

C:\Users\Lorenzo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\cluster\_agglomerative.py:1006: FutureWarning: Attribute `affinity` was deprecated in version 1.2 and will be removed in 1.4. Use `metric` instead
  warnings.warn(


,Hold_out_proteins,Cluster_Labels
0,Pr/PI,0
1,Pr/PI,0
2,Pr/PI,0
3,Pr/PI,0
4,Pr/PI,0
...,...,...
5107,"TCR/pMHC,1JCK_A_B",1
5108,"TCR/pMHC,1JCK_A_B",2
5109,"TCR/pMHC,1JCK_A_B",1
5110,"TCR/pMHC,1JCK_A_B",2


In [15]:
# Number of clusters
num_clusters = 5

# Iterate through each cluster
for i in range(num_clusters):
    # Filter the dataframe for the current cluster
    cluster_data = skempi_data_filtered[skempi_data_filtered['Cluster_Labels'] == i]
    
    # Print the details of the current cluster
    print(f"Cluster {i}:")
    print(cluster_data[['Hold_out_proteins', 'Cluster_Labels']])
    print("\n")


Cluster 0:
     Hold_out_proteins  Cluster_Labels
0                Pr/PI               0
1                Pr/PI               0
2                Pr/PI               0
3                Pr/PI               0
4                Pr/PI               0
...                ...             ...
5076          2KSO_A_B               0
5077          2KSO_A_B               0
5078          2KSO_A_B               0
5079          2KSO_A_B               0
5081    AB/AG,1UUZ_A_D               0

[4294 rows x 2 columns]


Cluster 1:
                                      Hold_out_proteins  Cluster_Labels
156                                   1SBB_A_B,1JCK_A_B               1
157                                   1SBB_A_B,1JCK_A_B               1
158                                   1SBB_A_B,1JCK_A_B               1
159   1JCK_A_B,1SBB_A_B,3QDJ_ABC_DE,3QDG_ABC_DE,4L3E...               1
161   1JCK_A_B,1SBB_A_B,3QDJ_ABC_DE,3QDG_ABC_DE,4L3E...               1
...                                                

In [16]:
from sklearn.cluster import AgglomerativeClustering

# Perform Agglomerative Clustering with 10 clusters
clustering = AgglomerativeClustering(n_clusters=10, affinity='precomputed', linkage='complete')
labels = clustering.fit_predict(distance_matrix)

# Add the cluster labels to your dataframe
skempi_data_filtered['Cluster_Labels'] = labels

# Display the data with the cluster labels
print(skempi_data_filtered[['Hold_out_proteins', 'Cluster_Labels']])


C:\Users\Lorenzo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\cluster\_agglomerative.py:1006: FutureWarning: Attribute `affinity` was deprecated in version 1.2 and will be removed in 1.4. Use `metric` instead
  warnings.warn(


      Hold_out_proteins  Cluster_Labels
0                 Pr/PI               0
1                 Pr/PI               0
2                 Pr/PI               0
3                 Pr/PI               0
4                 Pr/PI               0
...                 ...             ...
5107  TCR/pMHC,1JCK_A_B               1
5108  TCR/pMHC,1JCK_A_B               2
5109  TCR/pMHC,1JCK_A_B               1
5110  TCR/pMHC,1JCK_A_B               2
5111  TCR/pMHC,1JCK_A_B               1

[4952 rows x 2 columns]


In [17]:
# Number of clusters
num_clusters = 10

# Iterate through each cluster
for i in range(num_clusters):
    # Filter the dataframe for the current cluster
    cluster_data = skempi_data_filtered[skempi_data_filtered['Cluster_Labels'] == i]
    
    # Print the details of the current cluster
    print(f"Cluster {i}:")
    print(cluster_data[['Hold_out_proteins', 'Cluster_Labels']])
    print("\n")

Cluster 0:
               Hold_out_proteins  Cluster_Labels
0                          Pr/PI               0
1                          Pr/PI               0
2                          Pr/PI               0
3                          Pr/PI               0
4                          Pr/PI               0
...                          ...             ...
5040  5XCO_A_B,5TAR_A_B,2A9K_A_B               0
5052  5TAR_A_B,5XCO_A_B,1HE8_A_B               0
5054  5TAR_A_B,5XCO_A_B,1HE8_A_B               0
5056  5TAR_A_B,5XCO_A_B,1HE8_A_B               0
5058  5TAR_A_B,5XCO_A_B,1HE8_A_B               0

[3494 rows x 2 columns]


Cluster 1:
                                      Hold_out_proteins  Cluster_Labels
156                                   1SBB_A_B,1JCK_A_B               1
157                                   1SBB_A_B,1JCK_A_B               1
158                                   1SBB_A_B,1JCK_A_B               1
159   1JCK_A_B,1SBB_A_B,3QDJ_ABC_DE,3QDG_ABC_DE,4L3E...               1
161